In [2]:
import pickle 
import requests 
import json
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [3]:
api_key = 'key'
current_version = requests.get('https://ddragon.leagueoflegends.com/api/versions.json').json()[0]
current_version

'10.22.1'

In [4]:
# 챔피언 종류 받아와서 아이디, key 따로 저장하기
champion_dict = {}
r = requests.get('http://ddragon.leagueoflegends.com/cdn/{}/data/ko_KR/champion.json'.format(current_version))
info_df = pd.DataFrame(r.json())
for i, champ in enumerate(info_df.data):
    champion_dict[champ['id']] = champ['key']

In [5]:
# 챌~마스터 구간 유저 4300명 값 불러와서 dataframe으로 만들고 필요없는 컬럼들 제거하기

api_challenger = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
api_grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
api_master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key

league_df = pd.DataFrame(requests.get(api_challenger).json())
r = requests.get(api_grandmaster)
league_df = pd.concat([league_df, pd.DataFrame(r.json())], axis=0)
r = requests.get(api_master)
league_df = pd.concat([league_df, pd.DataFrame(r.json())], axis=0)

league_df.reset_index(inplace=True)
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T
league_df = pd.concat([league_df, league_entries_df], axis=1)
league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank', 'freshBlood', 'hotStreak', 'inactive', 'veteran', 'wins', 'losses'], axis=1)
league_df

,tier,leaguePoints,summonerId,summonerName
0,CHALLENGER,902,YAECZlT0gMbDeIkWn5VBAkVKULW9CzwqAep1fSNCPEmJyw,수기 김
1,CHALLENGER,1064,mFnLYwWsxr9WTQbCW0Iu2BfhjSO9AvxU2cyJeiUjwOjR1R8,매일배고프다
2,CHALLENGER,882,YTBlavPsuWU77_Cb7lAvO8zwzkT_t1zXC8DkG50gGXHbSZo,tyzj
3,CHALLENGER,838,EeHgEKZzMvVlLYcOt6Y2vLPfv6qS5y1xtBOMetoeRNL_wgF7,불면증환자임
4,CHALLENGER,1051,EEsreOggxSovcxK4vwqWERDl6eFi7H6DNg0soeRazgNdTGA,타 잔
5,CHALLENGER,971,dkjADc3yaSYJcFqV9UlBwmb237FGsR28vAjX-5tNb8LAADM,KTA MID
6,CHALLENGER,881,Z4Gu06PzsabNGwxCgTBANBbapphGseg9aDWJ7M0nKTWcceY,Envyzz
7,CHALLENGER,1054,WrS3IznrQi3o3BQuFzGnR-Ygv1Y7TyStXdZ-r0g8KahGpb0,SANDBOX FATE
8,CHALLENGER,980,xtJBtIEe0wXs8dhNaEEPB3B1HCmCrUEfzbus4nqXxpGg51M,T1 Cuzz
9,CHALLENGER,1428,S6M62Yd79A96eYZGLsJXJ0Q11IcSA1y9xqCPznuugZx4pHg,잇츠 낫배드


In [6]:
# summonerID값으로 accountID 불러와서 df에 합치기

league_df['account_id'] = np.nan
for i, summoner_id in enumerate(league_df[:20]['summonerId']):
    api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200:
        time.sleep(5)
        r = requests.get(api_url)
    account_id = r.json()['accountId']
    league_df.iloc[i, -1] = account_id

In [7]:
league_df

,tier,leaguePoints,summonerId,summonerName,account_id
0,CHALLENGER,902,YAECZlT0gMbDeIkWn5VBAkVKULW9CzwqAep1fSNCPEmJyw,수기 김,Fe7sW62-hqr-VJxxtNV8R6ZrfXWze1kUPiJl-FQHAFi5
1,CHALLENGER,1064,mFnLYwWsxr9WTQbCW0Iu2BfhjSO9AvxU2cyJeiUjwOjR1R8,매일배고프다,i30QAALDUrwSriuiHTv7lOWb_SOXkvuf7vCW_KaQGlvC8W...
2,CHALLENGER,882,YTBlavPsuWU77_Cb7lAvO8zwzkT_t1zXC8DkG50gGXHbSZo,tyzj,rKQthGhFsEAykmgXvXHK_n6E1-AZ79hFcVZfg05s0_t_XN...
3,CHALLENGER,838,EeHgEKZzMvVlLYcOt6Y2vLPfv6qS5y1xtBOMetoeRNL_wgF7,불면증환자임,dQDS8inltXeluy9cJIrO0A4JgdMbtgo0P4pwXsiTAimRoK...
4,CHALLENGER,1051,EEsreOggxSovcxK4vwqWERDl6eFi7H6DNg0soeRazgNdTGA,타 잔,qBp8jNLncDxENX8EzBrQWrMh6yhhn8Goy-45PkVFbCN23BU
5,CHALLENGER,971,dkjADc3yaSYJcFqV9UlBwmb237FGsR28vAjX-5tNb8LAADM,KTA MID,zZarphZ0adxxGfqHzeqQmsrwXlDGaFnAdCIeA0bCSk5vdc...
6,CHALLENGER,881,Z4Gu06PzsabNGwxCgTBANBbapphGseg9aDWJ7M0nKTWcceY,Envyzz,_AGB9b4rCQHlnjG3D2Ddm-wgud_tOYro2RwkReRSbeFQhW...
7,CHALLENGER,1054,WrS3IznrQi3o3BQuFzGnR-Ygv1Y7TyStXdZ-r0g8KahGpb0,SANDBOX FATE,5CiIz8K0EvhYH0xNAN655woltN0KblL0_tU3eeYTYcq-UQs
8,CHALLENGER,980,xtJBtIEe0wXs8dhNaEEPB3B1HCmCrUEfzbus4nqXxpGg51M,T1 Cuzz,HXoUci0Lyr7EmE-2EHax7n9Vv-LZgfL1i4hD_bajbXU927M
9,CHALLENGER,1428,S6M62Yd79A96eYZGLsJXJ0Q11IcSA1y9xqCPznuugZx4pHg,잇츠 낫배드,KqfZBLlkBVieFTzIFCmztazm09Y_QB3xega0mBwzrjcnLt...


In [8]:
# accountID로 순서대로 1계정당 100개씩 전적 불러오기 (테스트로 20개)

season = str(13)
match_info_df = pd.DataFrame()
for account_id in league_df[:20]['account_id']:
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + account_id + \
                  '?season=' + season + '&api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200:
        time.sleep(5)
        r = requests.get(api_url)
    match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])

In [9]:
match_info_df

,champion,gameId,lane,platformId,queue,role,season,timestamp
0,3,4767634967,MID,KR,420,SOLO,13,1604738890301
1,61,4767535900,MID,KR,420,SOLO,13,1604736224766
2,246,4759933757,NONE,KR,420,DUO_SUPPORT,13,1604408685385
3,117,4759813535,MID,KR,420,DUO,13,1604405982924
4,117,4759764688,MID,KR,420,SOLO,13,1604403889892
5,117,4748293073,MID,KR,420,SOLO,13,1603966966987
6,117,4745470342,TOP,KR,420,SOLO,13,1603820500594
7,117,4745415657,MID,KR,420,SOLO,13,1603817865401
8,3,4745410979,TOP,KR,420,SOLO,13,1603815339521
9,117,4745274035,MID,KR,420,SOLO,13,1603811873964


In [10]:
# gameID가 겹치면 삭제. 상위티어이기 때문에 약 5%가량 발생

match_info_df = match_info_df.drop_duplicates('gameId')

In [11]:
match_info_df

,champion,gameId,lane,platformId,queue,role,season,timestamp
0,3,4767634967,MID,KR,420,SOLO,13,1604738890301
1,61,4767535900,MID,KR,420,SOLO,13,1604736224766
2,246,4759933757,NONE,KR,420,DUO_SUPPORT,13,1604408685385
3,117,4759813535,MID,KR,420,DUO,13,1604405982924
4,117,4759764688,MID,KR,420,SOLO,13,1604403889892
5,117,4748293073,MID,KR,420,SOLO,13,1603966966987
6,117,4745470342,TOP,KR,420,SOLO,13,1603820500594
7,117,4745415657,MID,KR,420,SOLO,13,1603817865401
8,3,4745410979,TOP,KR,420,SOLO,13,1603815339521
9,117,4745274035,MID,KR,420,SOLO,13,1603811873964


In [12]:
# 위에서 얻어낸 게임ID를 가지고 실제 게임데이터를 받아옴

match_df = pd.DataFrame()
for game_id in match_info_df[:20]['gameId']:
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matches/' + str(game_id) + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200:
        time.sleep(5)
        r = requests.get(api_url)
    r_json = r.json()
    temp_df = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T
    match_df = pd.concat([match_df, temp_df])

In [13]:
match_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4767634967,KR,1604738890301,1976,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4767535900,KR,1604736224766,1650,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4759933757,KR,1604408685385,1070,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4759813535,KR,1604405982924,1361,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4759764688,KR,1604403889892,1523,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4748293073,KR,1603966966987,1602,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4745470342,KR,1603820500594,1888,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4745415657,KR,1603817865401,2051,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4745410979,KR,1603815339521,1503,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4745274035,KR,1603811873964,1265,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [14]:
# 게임 내 여러가지 테마의 게임이 있지만, 소환사의협곡 솔로랭크와 팀랭크만 수집

select_indices = (match_df['queueId']==420) | (match_df['queueId']==440)
match_df = match_df.loc[select_indices, :].reset_index(drop=True)
match_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4767634967,KR,1604738890301,1976,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,4767535900,KR,1604736224766,1650,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,4759933757,KR,1604408685385,1070,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,4759813535,KR,1604405982924,1361,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,4759764688,KR,1604403889892,1523,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5,4748293073,KR,1603966966987,1602,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
6,4745470342,KR,1603820500594,1888,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
7,4745415657,KR,1603817865401,2051,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
8,4745410979,KR,1603815339521,1503,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
9,4745274035,KR,1603811873964,1265,420,11,13,10.21.339.2173,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [15]:
# match_df에 3가지 team, participants, participantIdentities들이 문자열로 받아지는 현상 방지, 1회만 하면 됨

# for column in ['teams', 'participants', 'participantIdentities']:
#     match_df[column] = match_df[column].map(lambda v: eval(v))

In [16]:
# 테스트용으로 받아온 15경기의 챔피언 번호 받아오기

champion = []
for i in range(len(match_df)):
    champion = []
    for _ in range(10):
        # 탑 0, 5 정글 1, 6 미드 2, 7 원딜 3, 8 서폿 4, 9로 맞추기 위한 작업필요
        # 0~4, 5~9는 이긴 팀이 앞쪽으로 맞추기
        championNum = match_df['participants'][i][_]['championId']
        win = match_df['participants'][i][_]['stats']['win']
        lane = match_df['participants'][i][_]['timeline']['lane']
        champion.append(championNum)
    print(champion)

[22, 68, 64, 517, 80, 76, 202, 89, 3, 164]
[104, 80, 145, 14, 30, 89, 61, 67, 203, 81]
[432, 203, 81, 117, 517, 68, 22, 12, 246, 60]
[27, 523, 875, 517, 147, 111, 117, 164, 145, 120]
[497, 117, 98, 104, 81, 164, 201, 22, 41, 876]
[164, 421, 497, 117, 22, 61, 104, 517, 875, 523]
[517, 81, 421, 164, 235, 497, 117, 141, 202, 142]
[412, 81, 126, 30, 875, 497, 117, 203, 54, 523]
[126, 876, 202, 78, 517, 497, 3, 157, 120, 98]
[25, 202, 120, 13, 236, 497, 117, 523, 64, 24]
[412, 119, 43, 85, 104, 497, 117, 517, 81, 203]
[497, 117, 76, 29, 266, 235, 236, 876, 516, 202]
[119, 60, 517, 40, 126, 8, 21, 104, 497, 3]
[164, 203, 12, 61, 498, 202, 497, 3, 24, 64]
[51, 516, 25, 876, 236, 30, 164, 235, 497, 112]
[104, 517, 89, 98, 202, 24, 203, 497, 117, 22]
[84, 127, 89, 2, 202, 164, 20, 157, 497, 112]
[120, 164, 202, 43, 84, 22, 58, 5, 497, 117]
[89, 51, 30, 777, 164, 497, 117, 145, 141, 875]
[22, 85, 64, 497, 161, 142, 81, 117, 104, 122]
